In [1]:
import pandas as pd  # таблицы

from transformers import MBartTokenizer, MBartForConditionalGeneration
from itertools import combinations
import torch

from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
import networkx as nx


dir = "D:\Develop\VS\HSE\Pump_Dump\\"  # папка проекта
main_df = pd.read_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|", index_col=0)
main_df.head()
sum_df = main_df[(main_df["TICKERS"].str.len()>2) & (main_df["TICKERS"].str.len()<5) & (main_df["TICKERS"]=="lkoh")][["NEWS_TEXT", "TICKERS"]][:20]
sum_df


,NEWS_TEXT,TICKERS
256,Дивидендный сезон 2024. Последние майские отсе...,lkoh
421,"Так ждали дивиденды Лукойла, что не хочется ра...",lkoh
494,Дивидендный сезон: Топ 10 акций по дивидендной...,lkoh
682,Российский рынок. Куда дальше? Дивидендный сез...,lkoh
817,Мы скорректировали свой прогноз по ключевой ст...,lkoh
819,На этой короткой неделе курс еще может повторн...,lkoh
821,ПАО «Лукойл» Тикер: LKOH Идея: Long Срок идеи:...,lkoh
827,«Народный портфель». Частные инвесторы покупаю...,lkoh
882,Главное к открытию понедельник 06.05 : брифинг...,lkoh
908,Как быстро закроется майский дивидендный гэп Л...,lkoh


In [2]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

In [3]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [4]:
def gen_summary(article_text):
    print(article_text)
    input_ids = tokenizer(
        [article_text],
        max_length=600,
        truncation=True,
        return_tensors="pt",
    )["input_ids"].to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [5]:
sum_df["Summary_Bart"] = sum_df["NEWS_TEXT"].apply(gen_summary)

Дивидендный сезон 2024. Последние майские отсечки Формальной даты старта дивидендного сезона нет: его можно отсчитывать с начала мая или, например, с первой отсечки крупного эмитента, такого как ЛУКОЙЛ. В текущем сезоне отсечки обозначили уже более 60 эмитентов. Это не финальное число: некоторые компании еще не дали рекомендацию по выплате дивидендов, поэтому список будет дополняться. Со всеми предстоящими закрытиями реестра можно ознакомиться в дивидендном календаре. Топ 10 прошедших дивидендных отсечек и Топ 10 акций по дивидендной доходности: https://bcs express.ru/novosti i analitika/dividendnyi sezon 2024 poslednie maiskie otsechki
Так ждали дивиденды Лукойла, что не хочется расставаться? Понимаем. Но не унывайте: аналитики Альфа Инвестиций приготовили классные идеи, куда вложить выплаты 1 Акции Лукойла. Эксперты позитивно смотрят на них во II квартале и всём 2024 году. Как и на весь сектор в целом. До конца года цены на нефть, скорее всего, останутся высокими — растёт спрос, а пр

In [6]:
# Обрабатываем текст на основе PageRank

def similarity(s1, s2):
    if not len(s1) or not len(s2):
        return 0.0
    return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))

# Выдает список предложений отсортированных по значимости
def textrank(text):
    sentences = sent_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    lmtzr = RussianStemmer()
    words = [set(lmtzr.stem(word) for word in tokenizer.tokenize(sentence.lower()))
             for sentence in sentences] 	 
    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]
    scores = filter(lambda x: x[2], scores)
    g = nx.Graph()
    g.add_weighted_edges_from(scores)
    pr = nx.pagerank(g)
    return sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr), key=lambda x: pr[x[0]], reverse=True)

# Сокращает текст до нескольких наиболее важных предложений
def sumextract(text, n=5):
    tr = textrank(text)
    top_n = sorted(tr[:n])
    return ' '.join(x[2] for x in top_n)

In [7]:
sum_df["Summary_PageRank"] = sum_df["NEWS_TEXT"].apply(sumextract)

In [8]:
sum_df

,NEWS_TEXT,TICKERS,Summary_Bart,Summary_PageRank
256,Дивидендный сезон 2024. Последние майские отсе...,lkoh,Дивидендный сезон 2024. Последние майские отсе...,Дивидендный сезон 2024. Последние майские отсе...
421,"Так ждали дивиденды Лукойла, что не хочется ра...",lkoh,"В 2024 году Лукойл может заработать больше, че...",По итогам 2024 года Лукойл может заплатить 110...
494,Дивидендный сезон: Топ 10 акций по дивидендной...,lkoh,Дивидендный сезон: Топ 10 акций по дивидендной...,Дивидендный сезон: Топ 10 акций по дивидендной...
682,Российский рынок. Куда дальше? Дивидендный сез...,lkoh,Российский рынок в разгаре. Индекс Мосбиржи у ...,Дивидендный сезон в разгаре. Индекс Мосбиржи у...
817,Мы скорректировали свой прогноз по ключевой ст...,lkoh,В преддверии конца года Банк России может сниз...,Мы скорректировали свой прогноз по ключевой ст...
819,На этой короткой неделе курс еще может повторн...,lkoh,В преддверии закрытия дивидендного реестра «Ле...,На этой короткой неделе курс еще может повторн...
821,ПАО «Лукойл» Тикер: LKOH Идея: Long Срок идеи:...,lkoh,«Лукойл» — одна из крупнейших публичных вертик...,ПАО «Лукойл» Тикер: LKOH Идея: Long Срок идеи:...
827,«Народный портфель». Частные инвесторы покупаю...,lkoh,Московская биржа опубликовала данные о «Народн...,«Народный портфель». Частные инвесторы покупаю...
882,Главное к открытию понедельник 06.05 : брифинг...,lkoh,В ближайшие 5 дней на российском фондовом рынк...,"Подробнее об индексе здесь. 91.9 ₽/$, 12.7 ₽/ ..."
908,Как быстро закроется майский дивидендный гэп Л...,lkoh,"Понедельник, 6 мая 2024 г., станет последним д...",Как быстро закроется майский дивидендный гэп Л...


In [11]:
from datasets import load_metric
metric = load_metric("rouge", trust_remote_code=True)

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result


print("Bart - ",calc_rouge_scores(sum_df["NEWS_TEXT"], sum_df["Summary_Bart"]))
print("PageRank - ",calc_rouge_scores(sum_df["NEWS_TEXT"], sum_df["Summary_PageRank"]))

Bart -  {'rouge1': 35.6, 'rouge2': 22.8, 'rougeL': 34.8, 'rougeLsum': 34.7}
PageRank -  {'rouge1': 71.1, 'rouge2': 63.8, 'rougeL': 71.1, 'rougeLsum': 71.0}


In [10]:
sum_df[["NEWS_TEXT","Summary_Bart","Summary_PageRank"]].to_html(dir+"Answers\\compare.html")
# main_df.to_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|")